# IT-Skills for Research

In [ ]:
# Import packages
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd



In [4]:
# -*- coding: utf-8 -*-
"""
Created on Sun Nov 27 16:18:28 2022

@author: angel
"""
#%%
### Download necessary libraries
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import datetime 
from pandas.tseries.offsets import BDay
import matplotlib.pyplot as plt 



path = r"C:\Users\angel\Final--crypto\data"
path= "/Users/julianfischer/projects/Final--crypto/data"

def loadData(path):
    #cryptoPrices = pd.read_excel(path + '\CommonAssetClassesPrices.xlsx')
    equityPrices = pd.read_excel(path + '\Equities.xlsx')
    bondPrices = pd.read_excel(path + '\Bonds.xlsx')
    cmdtyPrices = pd.read_excel(path + '\Commodities.xlsx')
    vixPrices = pd.read_excel(path + '\VIX.xlsx')
    cryptoPrices = pd.read_excel(path + '\crypto-prices.xlsx')
    cryptoPrices['Dates'] = pd.to_datetime(cryptoPrices['Dates'])
    
    return equityPrices, bondPrices, cmdtyPrices, vixPrices, cryptoPrices

def loadData_forcoolguys(path):
    #cryptoPrices = pd.read_excel(path + '\CommonAssetClassesPrices.xlsx')
    equityPrices = pd.read_excel(path + '/Equities.xlsx')
    bondPrices = pd.read_excel(path + '/Bonds.xlsx')
    cmdtyPrices = pd.read_excel(path + '/Commodities.xlsx')
    vixPrices = pd.read_excel(path + '/VIX.xlsx')
    cryptoPrices = pd.read_excel(path + '/crypto-prices.xlsx')
    cryptoPrices['Dates'] = pd.to_datetime(cryptoPrices['Dates'])
    
    return equityPrices, bondPrices, cmdtyPrices, vixPrices, cryptoPrices

def getStartEnd(*data):
    minDate = datetime.datetime.strptime("30/11/0001","%d/%m/%Y")
    maxDate = datetime.datetime.strptime("30/11/3000","%d/%m/%Y")
    for d in data: 
        end = d['Dates'].iat[-1]
        start = d['Dates'][0]
        
        if end<maxDate:
            maxDate = end
            
        if start>minDate:
            minDate = start
            
    return minDate, maxDate
    
            
def makeDataUniformLength(start, end, assetList, *data):
    cnt = 0
    #aggData = []
    aggData = {}
    for d in data:
        d = d[(d['Dates']>=start)&(d['Dates']<=end)]
        #aggData.append([d])
        aggData[assetList[cnt]] = d
        cnt+=1
    
    return aggData

def mergeDataFrames(dictDf, assetList):
    merged = dictDf[assetList[0]]
    
    for i in range(len(assetList)):
        if i == len(assetList)-1:
            break
        merged = pd.merge(merged, dictDf[assetList[i+1]], on = ['Dates'])

    return merged 

def getBusinessDaysOnly(df):
    isBusinessDay = BDay().onOffset
    #isBusinessDay = BDay().is_on_offset()
    match_series = pd.to_datetime(df['Dates']).map(isBusinessDay)
    return df[match_series]

def calcDailyReturn(price):
    return price.pct_change(1)

def getCorr(df1,df2):
    corr, _ = pearsonr(df1, df2)
    return corr
    
#def plotRelationship(df1,df2):
#%%
assetNamesList = ['equity', 'bond', 'commodity', 'vix', 'crypto']
equity, bond, cmdty, vix, crypto = loadData_forcoolguys(path) 
start, end = getStartEnd(equity, bond, cmdty, vix, crypto)
allData = makeDataUniformLength(start, end, assetNamesList, equity, bond, cmdty, vix, crypto)
mergedData = mergeDataFrames(allData, assetNamesList)

dailyRets = {}
for a in assetNamesList:
    dailyRets[a] = calcDailyReturn(allData[a].iloc[:,1:])

mergedDailyRets = calcDailyReturn(mergedData.iloc[:,1:])
mergedDailyRets.index = mergedData.Dates

In [5]:
mergedDailyRets


,Hang Seng,NASDAQ,Stoxx 50,Nikkei,S&P500,GILTS,US 3m Treasury,US 5YR Treasury,US 10YR Treasury,Euro 10y,...,Copper,Platinum,Crude,Nat. gas,Liver cattle,VIX Index,BTC / USD Denominated Closing Price,ETH / USD Denominated Closing Price,XRP / USD Denominated Closing Price,XLM / USD Denominated Closing Price
Dates,,,,,,,,,,,,,,,,,,,,,
2015-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-10-01,0.000000,0.001496,-0.010198,0.019224,0.001974,-0.011351,0.251232,0.005968,0.000000,0.003699,...,-0.015591,-0.004288,-0.007673,-0.036064,-0.007070,-0.079592,0.006294,-0.082565,-0.032678,0.117503
2015-10-02,0.031650,0.017439,0.006233,0.000153,0.014315,-0.023536,-0.799213,-0.051124,-0.021553,0.003035,...,0.009104,0.004752,0.017796,0.007441,0.010581,-0.071397,-0.001229,0.015193,0.080795,-0.029315
2015-10-05,0.016201,0.015610,0.033097,0.015817,0.018290,0.049383,1.000000,0.036279,0.031763,-0.004683,...,0.012899,0.003733,0.015917,-0.000414,-0.008755,-0.066858,0.012710,-0.085606,-0.163169,-0.025028
2015-10-06,-0.001047,-0.006881,0.009309,0.010031,-0.003588,0.007283,-0.500000,-0.013333,-0.012012,-0.002099,...,-0.000221,0.023878,0.049021,0.008149,0.023035,-0.007165,0.024814,0.071359,0.097739,0.039251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-14,0.016972,-0.011225,0.004914,-0.010618,-0.008936,0.002978,0.000505,0.013257,0.010780,0.000144,...,-0.020442,-0.004913,-0.034029,0.009148,0.000330,0.053730,-0.016987,-0.031366,-0.017618,-0.039621
2022-11-15,0.041057,0.014486,0.007095,0.000955,0.008713,-0.021675,-0.009280,-0.023586,-0.021798,0.005702,...,-0.003522,-0.010261,0.012799,0.017017,-0.001979,0.034134,0.013597,0.007112,0.032417,0.015366
2022-11-16,-0.004723,-0.015385,-0.008253,0.001362,-0.008252,-0.044310,0.021234,-0.011629,-0.021143,0.007536,...,-0.012173,-0.006064,-0.014493,0.033151,0.003471,-0.017522,-0.011627,-0.027814,-0.030963,-0.027134
